In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install -q -U git+https://github.com/sbrugman/SDGym.git@v0.2.2-hw

In [ ]:
!pip install -q -U ../../

In [2]:
from timeit import default_timer as timer
from functools import partial
from random import choices
import logging

In [3]:
import sdgym
from sdgym import load_dataset
from sdgym import benchmark
from sdgym import load_dataset

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

In [5]:
import pgmpy

from pgmpy.models import BayesianModel
from pgmpy.estimators import TreeSearch, HillClimbSearch, BicScore, ExhaustiveSearch, BayesianEstimator
from pgmpy.sampling import BayesianModelSampling

In [6]:
import xgboost as xgb
from xgboost import XGBClassifier

In [7]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.isotonic import IsotonicRegression

In [8]:
from scipy import interpolate

In [9]:
from synthsonic.models.kde_utils import kde_smooth_peaks_1dim, kde_smooth_peaks
from synthsonic.models.kde_copula_nn_pdf import KDECopulaNNPdf

In [10]:
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
logging.basicConfig(level=logging.INFO)

In [12]:
dataset_name = 'adult'

In [13]:
data, categorical_columns, ordinal_columns = load_dataset(dataset_name)

INFO:sdgym.data:Loading dataset adult (all variables, -1 distinct values threshold)
INFO:sdgym.data:Columns to zero-code: []


In [14]:
data.shape

(22561, 15)

In [15]:
categorical_columns, ordinal_columns

([1, 5, 6, 7, 8, 9, 13, 14], [3])

In [16]:
for i in range(data.shape[1]):
    print (i, len(np.unique(data[:, i])))

0 72
1 9
2 16469
3 16
4 16
5 7
6 15
7 6
8 5
9 2
10 115
11 87
12 88
13 41
14 2


In [18]:
data[:, 0]

array([27., 27., 25., ..., 45., 51., 61.], dtype=float32)

# run sdgym

In [25]:
def KDECopulaNNPdf_Synthesizer(real_data, categorical_columns, ordinal_columns):
    all_features = list(range(real_data.shape[1]))
    numerical_features = list(set(all_features) - set(categorical_columns + ordinal_columns))
    data = np.float64(real_data)
    
    n_samples = data.shape[0]
    n_features = data.shape[1]
    
    clf = xgb.XGBClassifier(
        n_estimators=250,
        reg_lambda=1,
        gamma=0,
        max_depth=9
    )
#     clf = MLPClassifier(alpha=0.1, random_state=0, max_iter=1000, early_stopping=True)
    
    kde = KDECopulaNNPdf(
        use_KDE=False, 
        categorical_columns=categorical_columns+ordinal_columns,
        distinct_threshold=-1,
        n_uniform_bins=30,
        n_calibration_bins=100,
        test_size=0.25,
#        clf=clf,
#         ordering='mi',
    )
    kde = kde.fit(data)
    
#     X_gen, sample_weight = kde.sample(n_samples)
    X_gen = kde.sample_no_weights(n_samples, show_progress=True)
    
    X_gen[:, categorical_columns+ordinal_columns] = np.round(X_gen[:, categorical_columns+ordinal_columns])
    X_gen = np.float32(X_gen)
    
    print(X_gen[:, :5])

    return X_gen

In [26]:
def KDECopulaNNPdf_SynthesizerInteger(real_data, categorical_columns, ordinal_columns):
    """Census has integer only..."""
    data = KDECopulaNNPdf_Synthesizer(real_data, categorical_columns, ordinal_columns)
    data = np.round(data)
    
    print(data[:, :5])

    return data

In [27]:
from sdgym.synthesizers import (
    CLBNSynthesizer, CTGANSynthesizer, IdentitySynthesizer, IndependentSynthesizer,
    MedganSynthesizer, PrivBNSynthesizer, TableganSynthesizer, TVAESynthesizer,
    UniformSynthesizer, VEEGANSynthesizer)

all_synthesizers = [
    #IdentitySynthesizer,
    #IndependentSynthesizer,
    #KDECopulaNNPdf_Synthesizer,
    KDECopulaNNPdf_SynthesizerInteger,
]

In [28]:
scores = sdgym.run(synthesizers=all_synthesizers, datasets=[dataset_name], iterations=1)

INFO:sdgym.benchmark:0%|          | 0/1 [00:00<?, ?it/s]
INFO:sdgym.benchmark:Evaluating KDECopulaNNPdf_SynthesizerInteger on dataset adult; iteration 0; 333.16 MB
INFO:sdgym.data:Loading dataset adult (all variables, -1 distinct values threshold)
INFO:sdgym.data:Columns to zero-code: []
INFO:sdgym.benchmark:Running KDECopulaNNPdf_SynthesizerInteger on dataset adult; iteration 0; 326.98 MB
INFO:KDECopulaNNPdf:Processing 6 numerical and 9 categorical columns
INFO:KDECopulaNNPdf:Transforming numerical variables.
INFO:KDECopulaNNPdf:Configuring Bayesian Network (cat+num).
INFO:KDECopulaNNPdf:Configuring classifier.
INFO:KDECopulaNNPdf:Generating 250000 data points.

Generating for node: 6: 100%|██████████| 15/15 [00:04<00:00,  3.50it/s]
INFO:KDECopulaNNPdf:Fitting discriminative learner: selected 15 features.
INFO:KDECopulaNNPdf:Calibrating classifier.
INFO:KDECopulaNNPdf:Maximum weight found: 16.997403996129595
INFO:KDECopulaNNPdf:Generating 364888 data points.

Generating for node: 6: 1

[[4.5999687e+01 6.0000000e+00 2.0531917e+05 1.2000000e+01 1.2999737e+01]
 [3.1999300e+01 0.0000000e+00 1.8383322e+05 8.0000000e+00 9.0000210e+00]
 [3.9999992e+01 0.0000000e+00 2.7047491e+05 1.4000000e+01 1.4000123e+01]
 ...
 [5.1999718e+01 2.0000000e+00 1.8120944e+05 1.3000000e+01 1.3000277e+01]
 [1.8881336e+01 3.0000000e+00 2.7165388e+05 1.2000000e+01 1.0000226e+01]
 [3.3999790e+01 0.0000000e+00 2.6036802e+05 1.1000000e+01 9.9998980e+00]]
[[4.60000e+01 6.00000e+00 2.05319e+05 1.20000e+01 1.30000e+01]
 [3.20000e+01 0.00000e+00 1.83833e+05 8.00000e+00 9.00000e+00]
 [4.00000e+01 0.00000e+00 2.70475e+05 1.40000e+01 1.40000e+01]
 ...
 [5.20000e+01 2.00000e+00 1.81209e+05 1.30000e+01 1.30000e+01]
 [1.90000e+01 3.00000e+00 2.71654e+05 1.20000e+01 1.00000e+01]
 [3.40000e+01 0.00000e+00 2.60368e+05 1.10000e+01 1.00000e+01]]


INFO:sdgym.evaluate:Evaluating using binary classifier AdaBoostClassifier
INFO:sdgym.evaluate:Evaluating using binary classifier LogisticRegression
INFO:sdgym.evaluate:Evaluating using binary classifier MLPClassifier
/Users/maxbaak/opt/miniconda3/envs/synthsonic/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(
INFO:sdgym.benchmark:Finished KDECopulaNNPdf_SynthesizerInteger on dataset adult; iteration 0; 370.28 MB
INFO:sdgym.benchmark:100%|##########| 1/1 [00:29<00:00, 29.59s/it]


In [29]:
scores

,adult/accuracy,adult/f1,timestamp
VEEGANSynthesizer,0.718792,0.162142,2020-04-12 09:41:35.096775
CLBNSynthesizer,0.769225,0.305083,2020-10-17 09:46:54.494331
CTGAN,0.793808,0.608179,2020-10-17 09:46:54.494331
CTGANSynthesizer,0.797583,0.602407,2020-10-17 09:46:54.494331
CopulaGAN,0.785250,0.606638,2020-10-17 09:46:54.494331
GaussianCopulaCategorical,NaN,NaN,2020-10-17 09:46:54.494331
GaussianCopulaCategoricalFuzzy,0.765658,0.257748,2020-10-17 09:46:54.494331
GaussianCopulaOneHot,0.779675,0.198042,2020-10-17 09:46:54.494331
IdentitySynthesizer,0.824425,0.663006,2020-10-17 09:46:54.494331
IndependentSynthesizer,0.638117,0.151106,2020-10-17 09:46:54.494331


In [30]:
scores.tail(4)

,adult/accuracy,adult/f1,timestamp
TVAESynthesizer,0.803008,0.618866,2020-10-17 09:46:54.494331
TableganSynthesizer,0.791183,0.352537,2020-10-17 09:46:54.494331
UniformSynthesizer,0.494717,0.241641,2020-10-17 09:46:54.494331
KDECopulaNNPdf_SynthesizerInteger,0.801900,0.611553,2021-05-16 07:06:47.854656
